In [1]:
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import random
import matplotlib.pyplot as plt # data visualization library
import operator

In [2]:
m_cols = ['movie_id','movie title','release date','video release date','IMDb URL'
          ,'unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']
data_u_item = pd.read_csv('ml-100k/u.item',delimiter='|',names=m_cols,encoding='latin-1')

In [3]:
data_u_item.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
data_u_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
movie_id              1682 non-null int64
movie title           1682 non-null object
release date          1681 non-null object
video release date    0 non-null float64
IMDb URL              1679 non-null object
unknown               1682 non-null int64
Action                1682 non-null int64
Adventure             1682 non-null int64
Animation             1682 non-null int64
Childrens             1682 non-null int64
Comedy                1682 non-null int64
Crime                 1682 non-null int64
Documentary           1682 non-null int64
Drama                 1682 non-null int64
Fantasy               1682 non-null int64
Film-Noir             1682 non-null int64
Horror                1682 non-null int64
Musical               1682 non-null int64
Mystery               1682 non-null int64
Romance               1682 non-null int64
Sci-Fi                1682 non-null int64
Thriller 

In [5]:
#explore users
u_cols = ['user_id','age','gender','occupation','zip_code']
data_u_users = pd.read_csv('ml-100k/u.user',delimiter='|',names=u_cols,encoding='latin-1')

In [6]:
data_u_users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [7]:
#explore ratings
r_cols = ['user_id','movie_id','rating','timestamp']
data_rating_u1_base = pd.read_csv('ml-100k/u1.base',delimiter='\t',names=r_cols,encoding='latin-1')

In [8]:
data_rating_u1_base.head()

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [19]:
# merge movies and rating data
movies_ratings_data = data_u_item.merge(data_rating_u1_base,on = 'movie_id',how = 'inner')

# Q2, a

In [20]:
def movies_by_mean_ratings(movies_and_ratings_data):
    movies_mean_ratings = movies_and_ratings_data[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).mean()
    movies_mean_ratings = movies_mean_ratings.sort_values(['rating'],ascending=False)
    return movies_mean_ratings

In [21]:
print movies_by_mean_ratings(movies_ratings_data).head(3)

                                                            rating
movie_id movie title                                              
1189     Prefontaine (1997)                                    5.0
1653     Entertaining Angels: The Dorothy Day Story (1996)     5.0
1599     Someone Else's America (1995)                         5.0


# Q2, b

In [22]:
def movies_by_count_ratings(movies_and_ratings_data):
    movies_ratings_count = movies_and_ratings_data[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
    movies_ratings_count = movies_ratings_count.sort_values(['rating'],ascending=False)
    return movies_ratings_count

In [23]:
print movies_by_count_ratings(movies_ratings_data).head(3)

                                    rating
movie_id movie title                      
50       Star Wars (1977)              484
181      Return of the Jedi (1983)     422
258      Contact (1997)                402


# Q2, c

In [31]:
def movies_by_rating_percent(movies_and_ratings_data):
    count_rating = movies_and_ratings_data[['movie_id','movie title','rating']].groupby(['movie_id','movie title']).count()
    count_rating = count_rating.reset_index()
    
    count_rating_big_than_4 = movies_and_ratings_data[['movie_id','movie title','rating']][movie_ratings_data.rating > 4].groupby(['movie_id','movie title']).count()
    count_rating_big_than_4 = count_rating_big_than_4.reset_index()
    
    movies_by_rating_percent  = pd.DataFrame(columns = ['movie_id','movie title','rating_percent'])
    for index,row in count_rating_big_than_4.iterrows():
        movie_id = row.movie_id
        movie_title = row['movie title']
        movie_rate_count = row.rating

        total_rating = count_rating.loc[count_rating['movie_id'] == movie_id].rating.values[0]
        rating_percent = (float(movie_rate_count) / total_rating) * 100

        movie = pd.DataFrame([[movie_id, movie_title, int(rating_percent)]], columns = ['movie_id','movie title','rating_percent'])
        movies_by_rating_percent = movies_by_rating_percent.append(movie, ignore_index=True)

    movies_by_rating_percent = movies_by_rating_percent.sort_values(['rating_percent'],ascending=False)
    return movies_by_rating_percent

In [32]:
print movies_by_rating_percent(movies_ratings_data).head(3)

     movie_id                                        movie title  \
1074     1467               Saint of Fort Washington, The (1993)   
1113     1653  Entertaining Angels: The Dorothy Day Story (1996)   
1085     1500                          Santa with Muscles (1996)   

     rating_percent  
1074            100  
1113            100  
1085            100  


# Q2, d

In [33]:
import math
def new_rating_cala(mean,count):
    return (math.log10(count) + 1) * mean

In [36]:
def movies_by_new_ratings(movies_and_ratings_data):
    new_ratings  = pd.DataFrame(columns = ['movie_id','movie title','my_rate'])

    mean_ratings = movies_by_mean_ratings(movies_and_ratings_data).reset_index()
    count_ratings = movies_by_count_ratings(movies_and_ratings_data).reset_index()
    for index,row in count_ratings.iterrows():
        movie_id = row.movie_id
        movie_title = row['movie title']
        movie_rate_count = row.rating
        movie_rate_mean = mean_ratings.loc[mean_ratings['movie_id'] == movie_id].rating.values[0]

        movie = pd.DataFrame([[movie_id, movie_title, new_rating_cala(movie_rate_mean,movie_rate_count)]], columns = ['movie_id','movie title','my_rate'])
        new_ratings = new_ratings.append(movie)

    new_ratings = new_ratings.sort_values(['my_rate'],ascending=False)
    return new_ratings

In [37]:
print movies_by_new_ratings(movies_ratings_data).head(3)

  movie_id              movie title    my_rate
0       50         Star Wars (1977)  16.064099
0      318  Schindler's List (1993)  15.150769
0      127    Godfather, The (1972)  15.039946


# Q2, e

In [40]:
#merge movies ratings data with users
movie_ratings_data_and_users = movies_ratings_data.merge(data_u_users,on = 'user_id',how = 'inner')

## MANS

In [52]:
mans_rating = movie_ratings_data_and_users[['movie_id','movie title','rating']][movie_ratings_data_and_users.gender == 'M']

In [53]:
mans_rating.head()

,movie_id,movie title,rating
0,1,Toy Story (1995),5
1,2,GoldenEye (1995),3
2,3,Four Rooms (1995),4
3,4,Get Shorty (1995),3
4,5,Copycat (1995),3


## Q2, e, a

In [54]:
print movies_by_mean_ratings(mans_rating).head(3)

                                    rating
movie_id movie title                      
1500     Santa with Muscles (1996)     5.0
1189     Prefontaine (1997)            5.0
1656     Little City (1998)            5.0


## Q2, e, b

In [55]:
print movies_by_count_ratings(mans_rating).head(3)

                                    rating
movie_id movie title                      
50       Star Wars (1977)              359
181      Return of the Jedi (1983)     318
100      Fargo (1996)                  300


## Q2, e, c

In [56]:
print movies_by_rating_percent(mans_rating).head(3)

c:\python27\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


     movie_id                  movie title rating_percent
1270     1676      War at Home, The (1996)            100
1250     1547             Show, The (1995)            100
1015     1100  What Happened Was... (1994)            100


## Q2, e, d

In [57]:
print movies_by_new_ratings(mans_rating).head(3)

  movie_id                     movie title    my_rate
0       50                Star Wars (1977)  15.557252
0      127           Godfather, The (1972)  14.718533
0      174  Raiders of the Lost Ark (1981)  14.658116


## womans

In [66]:
woman_ratings = movie_ratings_data_and_users[['movie_id','movie title','rating']][movie_ratings_data_and_users.gender == 'F']

In [67]:
woman_ratings.head()

,movie_id,movie title,rating
135,1,Toy Story (1995),4
136,10,Richard III (1995),2
137,14,"Postino, Il (1994)",4
138,25,"Birdcage, The (1996)",4
139,100,Fargo (1996),5


## Q2, e, a

In [68]:
print movies_by_mean_ratings(woman_ratings).head(3)

                            rating
movie_id movie title              
1594     Everest (1998)        5.0
1150     Last Dance (1996)     5.0
1301     Stripes (1981)        5.0


## Q2, e, b

In [69]:
print movies_by_count_ratings(woman_ratings).head(3)

                                      rating
movie_id movie title                        
50       Star Wars (1977)                125
286      English Patient, The (1996)     125
288      Scream (1996)                   118


## Q2, e, c

In [70]:
print movies_by_rating_percent(woman_ratings).head(3)

c:\python27\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


     movie_id              movie title rating_percent
1010     1670           Tainted (1998)            100
966      1427            Drunks (1995)            100
944      1335  American Buffalo (1996)            100


## Q2, e, d

In [71]:
print movies_by_new_ratings(woman_ratings).head(3)

  movie_id                       movie title    my_rate
0       50                  Star Wars (1977)  13.353876
0      318           Schindler's List (1993)  12.971110
0       98  Silence of the Lambs, The (1991)  12.603435


## Conclusions:

### Are there differences in mean values between two populations?

In [72]:
mans_rating[['rating']].mean()

rating    3.529115
dtype: float64

In [73]:
woman_ratings[['rating']].mean()

rating    3.526187
dtype: float64

### What are the three most popular movies among women?

In [74]:
print movies_by_new_ratings(woman_ratings).head(3)

  movie_id                       movie title    my_rate
0       50                  Star Wars (1977)  13.353876
0      318           Schindler's List (1993)  12.971110
0       98  Silence of the Lambs, The (1991)  12.603435


### What are the three most popular movies among men?

In [77]:
print movies_by_new_ratings(mans_rating).head(3)

  movie_id                     movie title    my_rate
0       50                Star Wars (1977)  15.557252
0      127           Godfather, The (1972)  14.718533
0      174  Raiders of the Lost Ark (1981)  14.658116


### Who have the highest gap between men and women?

#### abs:

#### high in men:

#### hign in women:

#### common ground:

# Q3, a

### build model

In [95]:
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error

#Generic function for making a classification model and accessing performance:
def classification_model(model, train,test, predictors, outcome):
    #Fit the model:
    predictions = model.fit(train[predictors],train[outcome]).predict(test[predictors])

    #Print accuracy
    accuracy = metrics.accuracy_score(predictions,test[outcome])
    print("Accuracy : %s" % "{0:.3%}".format(accuracy))

    print "MEA: {}".format(mean_absolute_error(test[outcome], predictions))

### prepare train u1.base

In [98]:
mean_rank = data_rating_u1_base[['movie_id','rating']].groupby('movie_id').mean()
mean_rank = mean_rank.reset_index()
mean_rank.head()

,movie_id,rating
0,1,3.892950
1,2,3.180952
2,3,3.000000
3,4,3.526316
4,5,3.304348


In [99]:
# merge rating and all movies, remove non helpfull columns.
mean_movie_ratings = data_u_item.merge(mean_rank,on = 'movie_id',how = 'inner')
mean_movie_ratings = mean_movie_ratings.drop(['movie title', 'video release date', 'IMDb URL'], axis=1)

In [100]:
mean_movie_ratings.head()

,movie_id,release date,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
0,1,01-Jan-1995,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3.892950
1,2,01-Jan-1995,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.180952
2,3,01-Jan-1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.000000
3,4,01-Jan-1995,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3.526316
4,5,01-Jan-1995,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,3.304348


In [102]:
# need to change value type. for rating and release date
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
mean_movie_ratings['rating'] = le.fit_transform(mean_movie_ratings['rating'].astype(int)) + 1
mean_movie_ratings['release date'] = le.fit_transform(mean_movie_ratings['release date'].astype(str))

### prepare u1.test

In [104]:
r_cols = ['user_id','movie_id','rating','timestamp']
test = pd.read_csv('ml-100k/u1.test',delimiter='\t',names=r_cols,encoding='latin-1')
test = test[['movie_id','rating']].groupby('movie_id').mean()
test = test.reset_index()
test.head()

,movie_id,rating
0,1,3.797101
1,2,3.307692
2,3,3.200000
3,4,3.657895
4,5,3.294118


In [105]:
# merge u1.test (ranking) with movies
test = data_u_item.merge(test,on = 'movie_id',how = 'inner')
test = test.drop(['movie title', 'video release date', 'IMDb URL'], axis=1)

In [106]:
test.head()

,movie_id,release date,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
0,1,01-Jan-1995,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3.797101
1,2,01-Jan-1995,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.307692
2,3,01-Jan-1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.200000
3,4,01-Jan-1995,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3.657895
4,5,01-Jan-1995,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,3.294118


In [107]:
le = LabelEncoder()
test['rating'] = le.fit_transform(test['rating'].astype(int)) + 1
test['release date'] = le.fit_transform(test['release date'].astype(str))

## AdaBoost

In [113]:
outcome_var = ['rating']
model = AdaBoostClassifier(n_estimators=20)
predictor_var = ['release date','unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']
classification_model(model, mean_movie_ratings,test,predictor_var,outcome_var)

Accuracy : 47.163%
MEA: 0.628368794326


# Q3, b

In [114]:
#for user in users:
    # a = all movie view by this user
    # remove a from factor 1 and get top 20
    # remove a from factor 2 and get top 20
    # remove a from factor 3 and get top 20
    # remove a from factor 4 and get top 20
    # remove a from all movies and select 20 random
    # calc recall precision for each method


# Q3, c

In [ ]:
#same in Q3,b just split to women and men